In [2]:
import pyreadr
import os 
import pandas as pd
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

df_hex = list(pyreadr.read_r('data/db_hex.rds').values())[0]


c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification\Py\notebooks


In [3]:
df = df_hex.sample(n=10000)

In [4]:
import re

# Ersetze NaN-Werte in den Spalten 'titel' und 'kursbeschreibung' durch leere Strings
df['titel'] = df['titel'].fillna('')
df['kursbeschreibung'] = df['kursbeschreibung'].fillna('')

# Kombiniere die Spalten und füge den Doppelpunkt nur hinzu, wenn eine Kursbeschreibung vorhanden ist
df['sentence'] = df.apply(lambda row: row['titel'] + (": " + row['kursbeschreibung'] if row['kursbeschreibung'] else ""), axis=1)

# # Entfernen der alphanumerischen Codes mit Punkten und Bindestrichen
# df['sentence'] = df['sentence'].str.replace(r'\b[\w]+([.-][\w]+)+\b', '', regex=True)

# # Entfernen von unnötigen Leerzeichen
# df['sentence'] = df['sentence'].str.strip().replace(r'\s+', ' ', regex=True)

In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
from tqdm import tqdm
tqdm.pandas()

In [7]:
import pandas as pd
from setfit import SetFitModel

# Laden des vortrainierten Modells von Hugging Face
#model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")
model = SetFitModel.from_pretrained("models")  # Ändere den Pfad zu deinem Modell


# Definiere die Labels in der richtigen Reihenfolge
fs_labels = ['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing']

# Definiere eine Funktion, die das Modell auf eine Kursbeschreibung anwendet
def predict_course_description(description):
    # Überprüfen, ob die Beschreibung ein String ist
    if isinstance(description, str):
        preds = model(description)
        return preds
    # Falls die Beschreibung keine gültige Zeichenfolge ist, gib eine leere Liste oder eine andere Standardrückgabe zurück
    return []

# Funktion, um Vorhersagen in String-Variable umzuwandeln
def convert_tensor_to_labels(tensor):
    # Falls der Tensor NaN ist, gib "Keine" zurück
    if pd.isna(tensor):
        return 'Keine'
    
    # Identifiziere die Positionen, wo der Wert 1 ist
    selected_labels = [fs_labels[i] for i, val in enumerate(tensor) if val == 1]
    
    # Falls keine Labels ausgewählt wurden, gib "Keine" zurück
    return ', '.join(selected_labels) if selected_labels else 'Keine'


# Wende die Vorhersagen und die Umwandlungsfunktion auf jede Kursbeschreibung an
df["Vorhersagen"] = df["sentence"].progress_apply(predict_course_description)
#df["FS_Skill"] = df["Vorhersagen"].progress_apply(lambda x: convert_tensor_to_labels(x))



# DataFrame mit den neuen FS_Skill-Spalten anzeigen
print(df)


c:\Users\Hueck\AppData\Local\pypoetry\Cache\virtualenvs\future-skill-classification--ys6-NbF-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10000/10000 [01:39<00:00, 100.17it/s]

            hochschulname_kurz  \
1165863              HU Berlin   
1755668   MLU Halle-Wittenberg   
74308                   U Köln   
1833104   MLU Halle-Wittenberg   
1592798               CAU Kiel   
...                        ...   
852822   FAU Erlangen-Nürnberg   
1805445   MLU Halle-Wittenberg   
760355   FAU Erlangen-Nürnberg   
1276319         HHU Düsseldorf   
806053   FAU Erlangen-Nürnberg   

                                                hochschule  \
1165863                     Humboldt-Universität zu Berlin   
1755668                                  Universität Halle   
74308                                  Universität zu Köln   
1833104                                  Universität Halle   
1592798            Christian-Albrechts-Universität zu Kiel   
...                                                    ...   
852822   Friedrich-Alexander-Universität Erlangen-Nürnberg   
1805445                                  Universität Halle   
760355   Friedrich-Alexander-Univ

In [8]:
def convert_tensor_to_labels(tensor):
    # Falls der Tensor NaN ist, gib "Keine" zurück
    if pd.isna(tensor):
        return 'Keine'
    
    # Sicherstellen, dass die Länge des Tensors der Länge der fs_labels-Liste entspricht
    if len(tensor) != len(fs_labels):
        print(f"Warnung: Unerwartete Tensorgröße {len(tensor)}, erwartet: {len(fs_labels)}")
        return 'Fehlerhafte Vorhersage'
    
    # Identifiziere die Positionen, wo der Wert 1 ist
    selected_labels = [fs_labels[i] for i, val in enumerate(tensor) if val == 1]
    
    # Falls keine Labels ausgewählt wurden, gib "Keine" zurück
    return ', '.join(selected_labels) if selected_labels else 'Keine'

fs_labels = ['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing']
df["FS_Skill"] = df["Vorhersagen"].progress_apply(lambda x: convert_tensor_to_labels(x))


100%|██████████| 10000/10000 [00:00<00:00, 34879.55it/s]


In [9]:
type(df["Vorhersagen"])

pandas.core.series.Series

In [10]:
# Exportiere den DataFrame in eine Excel-Datei
# Exportiere den DataFrame in eine Excel-Datei
df.to_csv('output_large.csv', index=False)





In [1]:
label_counts = df['FS_Skill'].value_counts()

NameError: name 'df' is not defined

In [11]:
from setfit import SetFitModel

# Download from the 🤗 Hub
model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")
# Run inference
preds = model("Grundlagen der Fachdidaktik Pädagogik")


c:\Users\Hueck\AppData\Local\pypoetry\Cache\virtualenvs\future-skill-classification--ys6-NbF-py3.12\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hueck\.cache\huggingface\hub\models--Chernoffface--fs-setfit-multilable-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
 
# Greife auf die gespeicherten Labels zu
labels = model.labels  # oder model.config.labels, je nach Konfiguration

# Zeige die Labels an
print(labels)

# Verwende das Modell für Vorhersagen
preds = model("Softwareentwicklung")
preds

['Hardware-/Robotikentwicklung', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'Data Analytics & KI', 'Quantencomputing', 'IT-Architektur']


tensor([0, 1, 0, 0, 0, 0])